In [2]:

from api_library import StorageConnect
from api_library import ACConnect
import logging
import json



def add_tags(tag, blob_name):
    """
    Initializes BigQuery and ActiveCampaign connections.
    
    - Fetches qualifying emails from a GCP storage blob.
    - Retrieves associated ActiveCampaign contact IDs.
    - Adds the specified tag to each contact in ActiveCampaign.
    
    Args:
        tag (int): The ActiveCampaign tag ID.
        blob_name (str): The name of the blob containing email data.
    """
    
    try:
        # Init StorageConnect
        storage_conn = StorageConnect()
        
        # Specify the blob name
        blob = storage_conn.raw_data_bucket.blob(blob_name)
        
        # retrieve the blob
        downloaded_data = blob.download_as_text()
        
        if not downloaded_data:
            logging.error(f'Blob {blob_name} is empty in project {storage_conn.project_id}.')
            raise RuntimeError(f'Downloaded file is empty: {blob_name}')

        logging.info(f'Successfully retrieved data from blob: {blob_name}, project: {storage_conn.project_id}')
    
        try: 
            # Convert JSON string to list
            email_ls = json.loads(downloaded_data)
        except json.JSONDecodeError as e:
            logging.error(f'Failed to parse JSON from blob {blob_name}: {e}')
            raise ValueError(f'Invalid JSON format in blob {blob_name}') from e

        # Init ACConnect, passing through the tag id
        ac_conn = ACConnect(tag)
        
        # Fetch the ac_contact_id for each email --> add tag for the email
        for email in email_ls:
            id = ac_conn.fetch_contact_id(email)
            if id:
                ac_conn.add_tag(id)
            else:
                logging.warning(f'Skipping email {email} - No ActiveCampaign contact ID found')
                
    except Exception as e:
        logging.error(f'Unexpected error in add_tags function: {e}', exc_info=True)
        
        
add_tags(6636, 'sm_poor_fair_emails')


INFO:root:GCP credentials found in environment variable: C:\Users\edohner\OneDrive - Lyric Opera of Chicago\Desktop\Python Learning\airflow_test_project\gcp_service_account.json
INFO:root:GCP credentials JSON loaded successfully.
INFO:root:Using project ID from JSON credentials: dbt-test-449821
INFO:root:Successfully retrieved data from blob: sm_poor_fair_emails, project: dbt-test-449821
INFO:root:activecampaign_api_config.json found at C:\Users\edohner\OneDrive - Lyric Opera of Chicago\Desktop\Python Learning\airflow_test_project\api_library_package\api_library\ac_api\activecampaign_api_config.json
INFO:root:Successfully loaded sm_api_config.json
INFO:root:ac_contact_id fetched successfully: 314935
INFO:root:Adding "sm_poor_fair" tag to ac_contact_id: 314935
INFO:root:Successfully added 'sm_poor_fair' tag to contact ID: 314935
INFO:root:ac_contact_id fetched successfully: 40614
INFO:root:Adding "sm_poor_fair" tag to ac_contact_id: 40614
INFO:root:Successfully added 'sm_poor_fair' tag 